# This is a sample Jupyter Notebook

Below is an example of a code cell. 
Put your cursor into the cell and press Shift+Enter to execute it and select the next one, or click 'Run Cell' button.

Press Double Shift to search everywhere for classes, files, tool windows, actions, and settings.

To learn more about Jupyter Notebooks in PyCharm, see [help](https://www.jetbrains.com/help/pycharm/ipython-notebook-support.html).
For an overview of PyCharm, go to Help -> Learn IDE features or refer to [our documentation](https://www.jetbrains.com/help/pycharm/getting-started.html).

In [4]:
# NECESSARY PACKAGES AND LIBRARIES
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd


In [38]:
# IMPORT CSV FILE
data = pd.read_csv('phivolcs_earthquake_data.csv')
data = pd.DataFrame(data)
data

,Date_Time_PH,Latitude,Longitude,Depth_In_Km,Magnitude,Location
0,2016-01-01 00:40:00,17.34,120.30,023,3.0,015 km N 87° W of San Esteban (Ilocos Sur)
1,2016-01-01 05:06:00,14.65,123.12,017,3.3,054 km N 42° E of Paracale (Camarines Norte)
2,2016-01-01 13:24:00,09.76,125.46,012,2.4,005 km S 42° W of Surigao City
3,2016-01-01 15:01:00,17.30,120.27,026,2.9,018 km S 81° W of San Esteban (Ilocos Sur)
4,2016-01-01 20:27:00,08.89,126.28,024,3.0,004 km S 44° W of Cagwait (Surigao del Sur)
...,...,...,...,...,...,...
91673,2024-08-31 21:54:00,10.16,126.62,010,3.1,062 km N 76° E of Burgos (Surigao Del Norte)
91674,2024-08-31 22:11:00,13.85,120.54,163,3.0,010 km N 76° W of Calatagan (Batangas)
91675,2024-08-31 22:36:00,13.68,120.64,090,1.5,017 km S 03° E of Calatagan (Batangas)
91676,2024-08-31 22:49:00,13.76,120.16,008,1.7,010 km N 63° W of Looc (Occidental Mindoro)


In [39]:
# CLEANING DATA
data = data[data["Latitude"].str.match(r'^\d')]
data = data[data["Longitude"].str.match(r'^\d')]
data = data[data["Depth_In_Km"].str.match(r'^\d')]
data = data[data["Location"].str.match(r'^\d')]


,Date_Time_PH,Latitude,Longitude,Depth_In_Km,Magnitude,Location
0,2016-01-01 00:40:00,17.34,120.30,023,3.0,015 km N 87° W of San Esteban (Ilocos Sur)
1,2016-01-01 05:06:00,14.65,123.12,017,3.3,054 km N 42° E of Paracale (Camarines Norte)
2,2016-01-01 13:24:00,09.76,125.46,012,2.4,005 km S 42° W of Surigao City
3,2016-01-01 15:01:00,17.30,120.27,026,2.9,018 km S 81° W of San Esteban (Ilocos Sur)
4,2016-01-01 20:27:00,08.89,126.28,024,3.0,004 km S 44° W of Cagwait (Surigao del Sur)
...,...,...,...,...,...,...
91673,2024-08-31 21:54:00,10.16,126.62,010,3.1,062 km N 76° E of Burgos (Surigao Del Norte)
91674,2024-08-31 22:11:00,13.85,120.54,163,3.0,010 km N 76° W of Calatagan (Batangas)
91675,2024-08-31 22:36:00,13.68,120.64,090,1.5,017 km S 03° E of Calatagan (Batangas)
91676,2024-08-31 22:49:00,13.76,120.16,008,1.7,010 km N 63° W of Looc (Occidental Mindoro)
